Run the notebook at TODAY after 15:00:00 (so the market is closed and data is available).

Input your current position.

The orders are for tomorrow.

In [1]:
import pandas as pd
cal = pd.read_csv('/data/linq/.qlib/qlib_data/cn_data/calendars/day.txt')
TODAY = str(cal.iloc[-1,0])
YESTERDAY = str(cal.iloc[-2,0])
# TODAY = str(cal.iloc[-2,0])
# YESTERDAY = str(cal.iloc[-3,0])

BT_START = '2024-12-01'

print(BT_START)
print(YESTERDAY)
print(TODAY)

2024-12-01
2024-12-11
2024-12-12


In [2]:
# BT_START = '2024-12-01'
# TODAY = '2024-12-09'

test_split = (TODAY, TODAY)
bt_split = (BT_START, YESTERDAY)
pos_txt = f'''\
date {TODAY}
cash 316
SH600570 800
SH603986 300
SZ002459 1300
SZ002129 2000
SZ000002 2300
SH600104 1100
SH600588 1500
SH600233 1000
SZ000999 300
'''



REC_LIST = [
    ('GBDT0', 'EXP_BENCH', 'c745fed5a5ec439f9abeafac76e750a5'), # GBDT BENCH_NOW r0 all
    # ('GBDT1', 'EXP_BENCH', 'ffe9470bec09417fb4db91128edc0a87'), # GBDT BENCH_NOW r1 all
    ('GBDT2', 'EXP_BENCH', 'e420ff30870d4e55a1c5e0fb207fa8a0'), # GBDT BENCH_A r0 all
    # ('GBDT3', 'EXP_BENCH', '1dcc7b1ec14e46379690c4f2d6e5242d'), # GBDT BENCH_A r1 all
    ('GBDT4', 'EXP_BENCH', '8d7e72af7f4b475f9f3646021a29bb1e'), # GBDT BENCH_B r0 all
    # ('GBDT5', 'EXP_BENCH', 'd308acba0eaf453b9f5acfc309b05c46'), # GBDT BENCH_B r1 all
    ('GBDT6', 'EXP_BENCH', 'ec4284016f6d44d897969410a5ef3ec0'), # GBDT BENCH_NOW r0 csi300
    # ('GBDT7', 'EXP_BENCH', 'c9dc8e870efc459396ea77850a7bc099'), # GBDT BENCH_NOW r1 csi300
    ('GBDT8', 'EXP_BENCH', '1f1711ba5caa4fd7a298196fd6a57558'), # GBDT BENCH_A r0 csi300
    # ('GBDT9', 'EXP_BENCH', 'a79446d3724942c58ed14c59622fbe2c'), # GBDT BENCH_A r1 csi300
    ('GBDT10', 'EXP_BENCH', '74d88afd5cb3448d831402b4e7571dc6'), # GBDT BENCH_B r0 csi300
    # ('GBDT5', 'EXP_BENCH', '58317fb6231c4ba0b4d8cecc07db822d'), # GBDT BENCH_B r1 csi300
]

# EXP_NAME, rid = REC_LIST[0]
TOPK = 5
NDROP = 1
HT = 0

info = {
    'ALGO': ['GBDT'],
    'market': ['csi300_ext'], 
    # 'market': ['all'], 
    'benchmark': ["SH000300"], 
    'feat': ["Alpha158"], 
    # 'feat': ["Alpha360"], 
    'label': ['r0'],
    'params': [f'topk {TOPK} ndrop {NDROP} HT {HT}']
}

In [3]:
import qlib
import pandas as pd
from pandas import Timestamp
import json
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.filter import NameDFilter
from qlib.backtest.position import Position
from qlib.data import D

import copy
import pprint
import warnings
warnings.filterwarnings('ignore')

provider_uri = "/data/linq/.qlib/qlib_data/cn_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

from lilab.qlib.utils.tools import normalize_position_history, load_position_text, load_position_history, save_position_history, fill_price_position_history
from lilab.qlib.backtest.benchmark import BENCH_Step

/home/linq/anaconda3/envs/qlib39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[3486866:MainThread](2024-12-12 21:25:25,819) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[3486866:MainThread](2024-12-12 21:25:25,823) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[3486866:MainThread](2024-12-12 21:25:25,825) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data/linq/.qlib/qlib_data/cn_data')}


Record position history

In [4]:
cur_pos = load_position_text(pos_txt)
norm_cur_pos = normalize_position_history(cur_pos)
norm_cur_pos = fill_price_position_history(norm_cur_pos)
try:
    pos_history = load_position_history()
except:
    pos_history = {}
pos_history.update(norm_cur_pos)
save_position_history(pos_history)
dt, pos = next(iter(cur_pos.items()))
account = pos.position
dt, norm_pos = next(iter(norm_cur_pos.items()))
norm_account = pos.position
hold_lst = pos.get_stock_list()
print(dt)
print(account)
print(norm_pos)
print(hold_lst)

2024-12-12 09:00:00
{'SH600570': {'amount': 800}, 'SH603986': {'amount': 300}, 'SZ002459': {'amount': 1300}, 'SZ002129': {'amount': 2000}, 'SZ000002': {'amount': 2300}, 'SH600104': {'amount': 1100}, 'SH600588': {'amount': 1500}, 'SH600233': {'amount': 1000}, 'SZ000999': {'amount': 300}, 'cash': 316.0}
{'_settle_type': 'None', 'position': {'SH600570': {'amount': 3727, 'price': 6.821974754333496}, 'SZ002129': {'amount': 4184, 'price': 4.798587799072266}, 'SH603986': {'amount': 1783, 'price': 14.21588134765625}, 'SZ000999': {'amount': 3640, 'price': 3.8244364261627197}, 'SH600233': {'amount': 7140, 'price': 2.0756261348724365}, 'SZ002459': {'amount': 17480, 'price': 1.1809934377670288}, 'SH600588': {'amount': 7188, 'price': 2.604010820388794}, 'SH600104': {'amount': 12461, 'price': 1.537676215171814}, 'SZ000002': {'amount': 14867, 'price': 1.3443801403045654}, 'cash': 316.0, 'now_account_value': 178415.90483009815}, 'init_cash': 316.0}
['SH600570', 'SZ002129', 'SH603986', 'SZ000999', 'SH6

Prepare dataset

In [5]:
nameDFilter = NameDFilter(name_rule_re='(SH60[0-9]{4})|(SZ00[0-9]{4})')
filter_pipe=[nameDFilter]
benchmark = BENCH_Step(test_split,
                        market=info['market'][0], \
                        benchmark=info['benchmark'][0], \
                        feat=info['feat'][0], \
                        label=info['label'][0], \
                        account=norm_account, \
                        filter_pipe=filter_pipe)

bt_benchmark = BENCH_Step(bt_split,
                        market=info['market'][0], \
                        benchmark=info['benchmark'][0], \
                        feat=info['feat'][0], \
                        label=info['label'][0], \
                        account=norm_account, \
                        filter_pipe=filter_pipe)

In [6]:
dataset = init_instance_by_config(benchmark.dataset_config)
bt_dataset = init_instance_by_config(bt_benchmark.dataset_config)
# dataset.prepare('test')

[3486866:MainThread](2024-12-12 21:26:13,248) INFO - qlib.timer - [log.py:127] - Time cost: 46.961s | Loading data Done
[3486866:MainThread](2024-12-12 21:26:13,254) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | DropnaLabel Done
[3486866:MainThread](2024-12-12 21:26:13,260) INFO - qlib.timer - [log.py:127] - Time cost: 0.005s | CSZScoreNorm Done
[3486866:MainThread](2024-12-12 21:26:13,261) INFO - qlib.timer - [log.py:127] - Time cost: 0.010s | fit & process data Done
[3486866:MainThread](2024-12-12 21:26:13,262) INFO - qlib.timer - [log.py:127] - Time cost: 46.976s | Init data Done
[3486866:MainThread](2024-12-12 21:27:00,581) INFO - qlib.timer - [log.py:127] - Time cost: 47.317s | Loading data Done
[3486866:MainThread](2024-12-12 21:27:00,587) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | DropnaLabel Done
[3486866:MainThread](2024-12-12 21:27:00,603) INFO - qlib.timer - [log.py:127] - Time cost: 0.015s | CSZScoreNorm Done
[3486866:MainThread](2024-12-12 21:27:00,604

In [7]:
label_df = bt_dataset.prepare("test")
label_df = pd.DataFrame(label_df.loc[:, 'LABEL0'])
label_df.columns = ['label']
# label_df.index = label_df.index.droplevel(0)
label_df

label
datetime   instrument          
2024-12-02 SH600000    0.016860
           SH600009   -0.000560
           SH600010   -0.005051
           SH600011    0.005666
           SH600015    0.030956
...                         ...
2024-12-11 SZ002841    0.039735
           SZ002916    0.004948
           SZ002920    0.009886
           SZ002938    0.007202
           SZ003816   -0.002463

[2056 rows x 1 columns]

Load and run model

In [22]:
def simp_eval(res_df, k, t=None, day=None):
    if day:
        date = str(cal.iloc[-day,0])
        res_df = res_df[res_df.index.get_level_values('datetime') >= date]
    # Step 1: Group by multiple indices
    grouped = res_df.groupby(['datetime'])

    # Step 2: Sort by 'value' within each group in descending order
    sorted_df = grouped.apply(lambda x: x.sort_values(by='score', ascending=False))
    sorted_df['win'] = sorted_df['label'] * sorted_df['score'] > 0
    # Step 3: Select the top 5 rows for each group
    top_k_per_group = sorted_df.groupby(['datetime']).head(k)
    if t:
        top_k_per_group = top_k_per_group.tail(t)
    # print(top_k_per_group)
    top_k_ic = top_k_per_group['label'].corr(top_k_per_group['score'])
    top_k_win = 1.0 * top_k_per_group['win'].sum() / len(top_k_per_group)
    return top_k_ic, top_k_win

def run_model(model):
    BT_EXP_NAME = 'backtest'

    # backtest and analysis
    with R.start(experiment_name=BT_EXP_NAME, uri='./mlrun'):    
        recorder = R.get_recorder()
        ba_rid = R.get_recorder().id
        sr = SignalRecord(model, bt_dataset, recorder)
        sr.generate()
    with R.start(experiment_name=BT_EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=ba_rid)
        bt_pred_df = recorder.load_object("pred.pkl")
    res_df = pd.concat([label_df, bt_pred_df], axis=1, sort=True).reindex( label_df.index)

    def eval_day(day=None):
        top5ic, top5win = simp_eval(res_df, 5, day=day)
        top10ic, top10win = simp_eval(res_df, 10, 5, day=day)
        # top15ic, top15win = simp_eval(res_df, 15, 5, day=day)
        top20ic, top20win = simp_eval(res_df, 20, 10, day=day)
        # top25ic, top25win = simp_eval(res_df, 25, 5, day=day)
        top30ic, top30win = simp_eval(res_df, 30, 10, day=day)
        top40ic, top40win = simp_eval(res_df, 40, 10, day=day)
        top50ic, top50win = simp_eval(res_df, 50, 10, day=day)
        topic, topwin = simp_eval(res_df, 50, day=day)
        # return [topic, topwin, top5ic, top5win, top10ic, top10win, top15ic, top15win, top20ic, top20win, top25ic, top25win, top30ic, top30win]
        return [topwin, top5win, top10win, top20win, top30win, top40win, top50win], [topic, top5ic, top10ic, top20ic, top30ic, top40ic, top50ic]
    day1metrics = eval_day(2)
    day5metrics = eval_day(6)
    day10metrics = eval_day(11)
    day20metrics = eval_day(21)
    EXP_NAME = 'realworld_position_maker'

    # backtest and analysis
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):    
        recorder = R.get_recorder()
        ba_rid = R.get_recorder().id
        sr = SignalRecord(model, dataset, recorder)
        sr.generate()
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=ba_rid)
        pred_df = recorder.load_object("pred.pkl")

    return day1metrics, day5metrics, day10metrics, day20metrics, pred_df

# Run

In [23]:
info_win_dict1, info_ic_dict1 = {}, {}
info_win_dict5, info_ic_dict5 = {}, {}
info_win_dict10, info_ic_dict10 = {}, {}
info_win_dict20, info_ic_dict20 = {}, {}
pred_lst = []

for NAME, EXP_NAME, rid in REC_LIST:
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=rid)
        model = recorder.load_object("trained_model")
    day1metrics, day5metrics, day10metrics, day20metrics, pred_df = run_model(model)
    info_win_dict1[NAME], info_ic_dict1[NAME] = day1metrics
    info_win_dict5[NAME], info_ic_dict5[NAME] = day5metrics
    info_win_dict10[NAME], info_ic_dict10[NAME] = day10metrics
    info_win_dict20[NAME], info_ic_dict20[NAME] = day20metrics
    # for k in [1, 5, 10, 20]:
    #     eval(f'    info_win_dict{k}[NAME], info_ic_dict{k}[NAME] = day{k}metrics')
    pred_lst.append(pred_df)

[3486866:MainThread](2024-12-12 21:36:28,701) INFO - qlib.workflow - [exp.py:258] - Experiment 2 starts running ...
[3486866:MainThread](2024-12-12 21:36:28,713) INFO - qlib.workflow - [recorder.py:341] - Recorder f88bfe7ad00d48228c05692009f2cb6d starts running under Experiment 2 ...
[3486866:MainThread](2024-12-12 21:36:28,872) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | waiting `async_log` Done
[3486866:MainThread](2024-12-12 21:36:28,878) INFO - qlib.workflow - [exp.py:258] - Experiment 4 starts running ...
[3486866:MainThread](2024-12-12 21:36:28,891) INFO - qlib.workflow - [recorder.py:341] - Recorder 88dbe794669e4978829e80db58d346a6 starts running under Experiment 4 ...
[3486866:MainThread](2024-12-12 21:36:29,050) INFO - qlib.workflow - [record_temp.py:198] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 4
[3486866:MainThread](2024-12-12 21:36:29,057) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


/home/linq/finance/private_trade/mlrun/1/c745fed5a5ec439f9abeafac76e750a5/artifacts/trained_model
trained_model
'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2024-12-02 SH600000   -0.007318
           SH600009   -0.005881
           SH600010    0.016440
           SH600011   -0.012365
           SH600015    0.002524


[3486866:MainThread](2024-12-12 21:36:29,063) INFO - qlib.workflow - [exp.py:258] - Experiment 4 starts running ...
[3486866:MainThread](2024-12-12 21:36:29,076) INFO - qlib.workflow - [recorder.py:341] - Recorder f041474359204893a91bdc4fcb80fb84 starts running under Experiment 4 ...
[3486866:MainThread](2024-12-12 21:36:29,201) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[3486866:MainThread](2024-12-12 21:36:29,363) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...
[3486866:MainThread](2024-12-12 21:36:29,381) INFO - qlib.workflow - [recorder.py:341] - Recorder 186667fc002049c2bc40868dcc405faf starts running under Experiment 3 ...


/home/linq/finance/private_trade/mlrun/4/88dbe794669e4978829e80db58d346a6/artifacts/pred.pkl
pred.pkl


[3486866:MainThread](2024-12-12 21:36:29,516) INFO - qlib.workflow - [record_temp.py:198] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 3
[3486866:MainThread](2024-12-12 21:36:29,527) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[3486866:MainThread](2024-12-12 21:36:29,534) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...
[3486866:MainThread](2024-12-12 21:36:29,550) INFO - qlib.workflow - [recorder.py:341] - Recorder 530a224f757b4a2a9ecf8fe1ef8e0946 starts running under Experiment 3 ...
[3486866:MainThread](2024-12-12 21:36:29,685) INFO - qlib.timer - [log.py:127] - Time cost: 0.001s | waiting `async_log` Done
[3486866:MainThread](2024-12-12 21:36:29,693) INFO - qlib.workflow - [exp.py:258] - Experiment 2 starts running ...
[3486866:MainThread](2024-12-12 21:36:29,702) INFO - qlib.workflow - [recorder.py:341] - Recorder 608f513ed2334c88bcce86576db3b244 starts running under Experiment 2 ...


...

/home/linq/finance/private_trade/mlrun/1/74d88afd5cb3448d831402b4e7571dc6/artifacts/trained_model
trained_model
'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2024-12-02 SH600000   -0.038353
           SH600009    0.004608
           SH600010    0.032711
           SH600011    0.007540
           SH600015    0.040062


[3486866:MainThread](2024-12-12 21:36:34,088) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[3486866:MainThread](2024-12-12 21:36:34,252) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...
[3486866:MainThread](2024-12-12 21:36:34,270) INFO - qlib.workflow - [recorder.py:341] - Recorder e4974bfb64ee4ab4952ac72ca67b6b2e starts running under Experiment 3 ...


/home/linq/finance/private_trade/mlrun/4/5741e3deb04f44e784d24eb93021285b/artifacts/pred.pkl
pred.pkl


[3486866:MainThread](2024-12-12 21:36:34,413) INFO - qlib.workflow - [record_temp.py:198] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 3
[3486866:MainThread](2024-12-12 21:36:34,419) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[3486866:MainThread](2024-12-12 21:36:34,426) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...
[3486866:MainThread](2024-12-12 21:36:34,442) INFO - qlib.workflow - [recorder.py:341] - Recorder 0509f87f83ab40f28eb646dd2324b60e starts running under Experiment 3 ...
[3486866:MainThread](2024-12-12 21:36:34,580) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | waiting `async_log` Done


'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2024-12-12 SH600000    0.030818
           SH600009    0.029509
           SH600010   -0.031014
           SH600011    0.009100
           SH600015   -0.049552
/home/linq/finance/private_trade/mlrun/3/e4974bfb64ee4ab4952ac72ca67b6b2e/artifacts/pred.pkl
pred.pkl


In [24]:
pred_df = pd.concat(pred_lst, axis=1)
pred_df.columns = name_lst = [r[0] for r in REC_LIST]
_pred_df = pred_df

pred_df['mean'] = _pred_df.mean(axis=1)
pred_df['std'] = _pred_df.std(axis=1)
pred_df['score'] = pd.concat([pred_df[n]*info_win_dict5[n][0] for n in name_lst], axis=1).sum(axis=1)
# pred_df

In [25]:
rank_df = pred_df.droplevel(0).sort_values(by='score', ascending=False)
fields = ['$close/$factor']
_df = D.features(rank_df.index.values, fields, start_time=TODAY, end_time=TODAY, freq='day')
_df.columns = ['close']
_df = _df.droplevel(1)
rank_df = pd.concat([rank_df,_df],axis=1,join='inner')
report_top = rank_df

In [26]:
hold_lst

['SH600570',
 'SZ002129',
 'SH603986',
 'SZ000999',
 'SH600233',
 'SZ002459',
 'SH600588',
 'SH600104',
 'SZ000002']

In [27]:
pos_rank = rank_df.sort_values(by='score', ascending=True)
pos_rank = pos_rank[pos_rank.index.isin(hold_lst)]
report_pos = pos_rank

In [28]:
color_lst = [n for n in report_pos.columns if n not in ['close']]
# color_lst

In [29]:
sty_report_pos = report_pos.style.background_gradient(cmap='Blues_r', subset=color_lst)
sty_report_pos

,GBDT0,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,
SH603986,0.060442,0.088996,0.138719,0.049820,0.069724,0.205405,0.102184,0.054316,-0.078136,84.529999
SZ002459,0.025286,0.062513,0.120249,0.053443,0.071341,0.138618,0.078575,0.038995,-0.062311,15.880000
SZ000002,0.060523,0.080939,0.119514,0.025535,0.049117,0.128262,0.077315,0.036851,-0.057456,8.690001
SH600570,0.044868,0.140914,0.094702,0.011965,0.043290,0.109030,0.074128,0.044297,-0.052846,31.790001
SZ002129,0.038912,0.059106,0.103770,0.044926,0.041208,0.085768,0.062282,0.024408,-0.047918,10.040000
SH600588,0.007686,0.026450,0.067287,-0.008885,0.000845,0.090871,0.030709,0.036435,-0.022981,12.480000
SH600233,-0.000358,-0.012536,-0.008486,-0.019066,-0.030283,-0.007431,-0.013027,0.009547,0.011079,14.820000
SZ000999,-0.030453,-0.044142,-0.068137,-0.039849,-0.040063,-0.056775,-0.046570,0.012399,0.036097,46.410000
SH600104,-0.045602,-0.095787,-0.118792,0.016905,-0.042704,-0.030952,-0.052822,0.044145,0.036754,17.420000


In [30]:
sty_report_top0 = report_top.head(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top1 = report_top.head(20).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top2 = report_top.head(30).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top3 = report_top.head(40).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top4 = report_top.head(50).tail(10).style.background_gradient(axis=0, subset=color_lst)

In [31]:
sty_report_top0

,GBDT0,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,
SH601336,-0.175025,-0.275242,-0.257120,-0.178139,-0.227061,-0.212558,-0.220857,0.037206,0.167165,52.490002
SZ002938,-0.119160,-0.185225,-0.192462,-0.175051,-0.226860,-0.330100,-0.204810,0.064474,0.160188,36.360001
SH603899,-0.143265,-0.250147,-0.144527,-0.101394,-0.190205,-0.271122,-0.183443,0.060613,0.136230,33.279999
SH600600,-0.168408,-0.177683,-0.166635,-0.212843,-0.221322,-0.105492,-0.175397,0.037702,0.135736,79.910004
SH600606,-0.085225,-0.124583,-0.172681,-0.152137,-0.149339,-0.334005,-0.169662,0.078421,0.134044,2.620000
SH603369,-0.120963,-0.155168,-0.174177,-0.170966,-0.207002,-0.179112,-0.167898,0.026041,0.131418,50.849998
SZ000596,-0.134033,-0.170283,-0.126781,-0.197224,-0.197149,-0.186589,-0.168677,0.028590,0.131266,205.219986
SZ000876,-0.116481,-0.155743,-0.112179,-0.164257,-0.210193,-0.205406,-0.160710,0.038286,0.125440,9.960000
SH600754,-0.123892,-0.171909,-0.150397,-0.135713,-0.174195,-0.201197,-0.159551,0.025901,0.122114,28.360001


In [32]:
sty_report_top1

,GBDT0,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,
SH600183,-0.118999,-0.158174,-0.156832,-0.104192,-0.156492,-0.189667,-0.147393,0.028130,0.111888,22.740002
SH605499,-0.138890,-0.074009,-0.121883,-0.205001,-0.137841,-0.144927,-0.137092,0.038487,0.108711,246.369980
SH600426,-0.123751,-0.153889,-0.156081,-0.096339,-0.137593,-0.179669,-0.141220,0.026433,0.106299,23.450001
SH600029,-0.102279,-0.128286,-0.154600,-0.067059,-0.145264,-0.110952,-0.118073,0.029099,0.089182,7.190000
SH603195,-0.074421,-0.108068,-0.137145,-0.065523,-0.092894,-0.196217,-0.112378,0.044115,0.085733,75.649994
SZ002603,-0.132777,-0.146696,-0.043211,-0.100952,-0.156481,-0.112079,-0.115366,0.037417,0.085563,17.640001
SH601059,-0.065753,-0.109138,-0.091047,-0.080024,-0.135270,-0.138162,-0.103232,0.026997,0.079947,16.360001
SH601888,-0.120594,-0.152394,-0.106380,-0.075910,-0.089788,-0.096879,-0.106991,0.024532,0.077611,73.810005
SZ002180,-0.121105,-0.175831,-0.100985,-0.076974,-0.092465,-0.058011,-0.104229,0.037504,0.074603,28.160000


In [33]:
sty_report_top2

,GBDT0,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,
SZ002415,-0.086471,-0.114292,-0.085118,-0.070417,-0.098074,-0.133884,-0.098043,0.020868,0.073539,31.579998
SH601877,-0.097392,-0.106734,-0.062746,-0.101819,-0.131512,-0.070887,-0.095182,0.022893,0.072567,23.040001
SH600050,-0.076851,-0.140671,-0.074943,-0.072334,-0.079562,-0.135794,-0.096693,0.029486,0.071671,5.330000
SZ000786,-0.056238,-0.060436,-0.043992,-0.108042,-0.116051,-0.134133,-0.086482,0.034177,0.069326,30.180000
SZ000807,-0.056096,-0.072321,-0.111642,-0.036449,-0.093762,-0.138354,-0.084771,0.034123,0.065051,14.970001
SH600132,-0.067145,-0.131565,-0.091725,-0.043766,-0.112054,-0.039172,-0.080904,0.034069,0.059745,67.650002
SH601111,-0.062203,-0.110572,-0.124704,-0.059308,-0.094236,-0.018987,-0.078335,0.035559,0.059116,8.790000
SZ002236,-0.061604,-0.079302,-0.047876,-0.055512,-0.086792,-0.118227,-0.074885,0.023526,0.056877,16.920000
SZ000625,-0.078348,-0.124782,-0.089282,-0.064163,-0.081618,-0.015339,-0.075589,0.032691,0.055335,14.620000


In [34]:
import dataframe_image as dfi

infodf = pd.DataFrame({'label': ['Model update date', 'Prediction generation date'],
                       'date': ['2024-12-11', TODAY]})

dfi.export(infodf, './tmp/ensemble_info.png',table_conversion='matplotlib')

# dfi.export(sty_report_info5, './tmp/ensemble_performance5.png',table_conversion='matplotlib')
# dfi.export(sty_report_info10, './tmp/ensemble_performance10.png',table_conversion='matplotlib')
# dfi.export(sty_report_info20, './tmp/ensemble_performance20.png',table_conversion='matplotlib')
dfi.export(sty_report_top0, './tmp/ensemble_top0.png',table_conversion='matplotlib')
dfi.export(sty_report_top1, './tmp/ensemble_top1.png',table_conversion='matplotlib')
dfi.export(sty_report_top2, './tmp/ensemble_top2.png',table_conversion='matplotlib')
dfi.export(sty_report_top2, './tmp/ensemble_top3.png',table_conversion='matplotlib')
dfi.export(sty_report_top2, './tmp/ensemble_top4.png',table_conversion='matplotlib')
dfi.export(sty_report_pos, './tmp/ensemble_watch.png',table_conversion='matplotlib')

In [35]:

for k in [1,5,10,20]:
    sty_report_win_info = eval(f"pd.DataFrame(info_win_dict{k}, index=['TOP50WIN', '0-5', '5-10', '10-20', '20-30', '30-40', '40-50'])").style.background_gradient(axis=0)
    sty_report_ic_info = eval(f"pd.DataFrame(info_ic_dict{k}, index=['TOP50IC', '0-5', '5-10', '10-20', '20-30', '30-40', '40-50'])").style.background_gradient(axis=0)
    dfi.export(sty_report_win_info, f'./tmp/ensemble_win{k}.png',table_conversion='matplotlib')
    dfi.export(sty_report_ic_info, f'./tmp/ensemble_ic{k}.png',table_conversion='matplotlib')